# Installations

In [1]:
!pip install turicreate --ignore-installed

     |████████████████████████████████| 92.0 MB 21 kB/s 
     |████████████████████████████████| 3.5 MB 33.2 MB/s 
     |████████████████████████████████| 27.4 MB 54.5 MB/s 
     |████████████████████████████████| 3.6 MB 64.1 MB/s 
     |████████████████████████████████| 3.0 MB 33.1 MB/s 
     |████████████████████████████████| 61 kB 4.9 MB/s 
     |████████████████████████████████| 322 kB 58.4 MB/s 
     |████████████████████████████████| 9.9 MB 53.9 MB/s 
     |████████████████████████████████| 15.3 MB 44.0 MB/s 
     |████████████████████████████████| 86.4 MB 85 kB/s 
     |████████████████████████████████| 1.0 MB 33.9 MB/s 
     |████████████████████████████████| 18.3 MB 19.5 MB/s 
     |████████████████████████████████| 821 kB 51.2 MB/s 
     |████████████████████████████████| 510 kB 60.1 MB/s 
     |████████████████████████████████| 227 kB 33.9 MB/s 
     |████████████████████████████████| 138 kB 52.0 MB/s 
     |████████████████████████████████| 178 kB 53.5 MB/s 
     |█████████

# Imports

In [2]:
import pandas as pd
import turicreate as tc
import glob
import datetime
from sklearn import preprocessing 


# Global Params

In [3]:
GLOBAL_PATH_TRAIN = "/kaggle/input/bgu-recsys-ctr-prediction/train_data/*"
TEST_PATH = "../input/bgu-recsys-ctr-prediction/test_file.csv"

# Data Exploration

In [4]:
def precnet_from_column(data,by,value):
    count_platform_clicks = data[[by,value]].groupby([value,by],{'count_click_per_class':tc.aggregate.COUNT()}).sort("count_click_per_class", ascending = False)
    count_platform_clicks_total = count_platform_clicks.groupby([by],{'count_total_obs':tc.aggregate.SUM("count_click_per_class")})
    count_platform_clicks = count_platform_clicks.join(count_platform_clicks_total,on = by)
    count_platform_clicks["count_click_precent"] = (1.0 *count_platform_clicks["count_click_per_class"]) / count_platform_clicks["count_total_obs"]
    return count_platform_clicks

In [5]:
## allows us to see all columns
pd.set_option('display.max_columns', None)
data = pd.read_csv("/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00045.csv")
## Show an example of the data
data.sample(3)

,page_view_start_time,user_id_hash,target_id_hash,syndicator_id_hash,campaign_id_hash,empiric_calibrated_recs,empiric_clicks,target_item_taxonomy,placement_id_hash,user_recs,user_clicks,user_target_recs,publisher_id_hash,source_id_hash,source_item_type,browser_platform,os_family,country_code,region,day_of_week,time_of_day,gmt_offset,is_click
134594,1586884106365,716fefde5cadc9b21fd59a90dc7b6c3db777f1276a523f...,dcf49550cf5b77473b2cb0f469a26769dee4b9de4885a6...,e4e85541c71dc9597b379e26645275b2c66edc43f71173...,0c85ca153e3f26ccb21e18166d7fc4790ef457649a60e5...,28063.52500,45241.0,LIFE~HEALTH,f1b3343d5e6491da91d7c65d1ad3140bb1983eedb1ed2d...,125.0,5.0,0.0,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,c1f6b23d13d56918520e22a77b899811f709027a98e404...,text,DESK,6,US,IN,2,13,-400,1.0
92364,1587094960372,f518f356c7eae9d2d36e2e4df572869fead227da949d79...,9cf5d31bb80cd748e0262be9c76b91f1578c3979e0fda6...,69a95437763a6d114f99d4be3c8d94201939ba220ddba6...,f7b78e12ef2be13e6185dbb8dde747cedd517b4699d254...,628201.25000,293304.0,LIFE~EDUCATION,6055091bd2f1c62e48ee3cb89c8e7f0829edc6a05e9f94...,152.0,0.0,22.0,f8a7ba9b7c9b05464ee98daac522b3f3d2376453c70e57...,389b69708cf58731e78e2e023a8c608d886fe5aebc0e23...,category,DESK,6,MN,1,5,11,800,0.0
385798,1586817253958,fde8b60eb88b0272373b22d80dd1bd1da54196b7558adb...,c4461944279d18bff25aaa894b9aa4e0e085950a3f58f6...,96ea90c879174d24c6e348904b05108a66a6aa87e175a0...,c2e4539930c6dd9d5dc15c80b36202a7c8b566c965bcec...,969.69904,2830.0,SPORTS~BASKETBALL,3e169c2c7c217891ced33c35c10f01ba9b1a830e58ff24...,562.0,21.0,0.0,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,10b4bd25f3a771d678e3e535a768d555b7da0eb8c3f84c...,text,DESK,6,US,WA,1,15,-700,0.0


### Read Data

This part will take time. **run only ones! and dont overwrite**

In [6]:
all_paths = glob.glob(GLOBAL_PATH_TRAIN)

data = tc.SFrame()

for path in all_paths:
    data = data.append(tc.SFrame.read_csv(path))

Finished parsing file /kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00045.csv

Parsing completed. Parsed 100 lines in 0.911045 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,str,str,float,float,str,str,float,float,float,str,str,str,str,int,str,str,int,int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 52860 lines. Lines per second: 58802.3

Finished parsing file /kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00045.csv

...

Finished parsing file /kaggle/input/bgu-recsys-ctr-prediction/train_data/train6/part-00055.csv

Read 4066688 lines. Lines per second: 48125.4

Read 4330983 lines. Lines per second: 48267

Finished parsing file /kaggle/input/bgu-recsys-ctr-prediction/train_data/train6/part-00059.csv

Parsing completed. Parsed 4347176 lines in 89.9069 secs.

In [7]:
test = tc.SFrame.read_csv(TEST_PATH)

Finished parsing file /kaggle/input/bgu-recsys-ctr-prediction/test_file.csv

Parsing completed. Parsed 100 lines in 1.56142 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,str,str,float,float,str,str,float,float,float,str,str,str,str,int,str,str,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 52684 lines. Lines per second: 33359.1

Read 315863 lines. Lines per second: 43047.5

Finished parsing file /kaggle/input/bgu-recsys-ctr-prediction/test_file.csv

Parsing completed. Parsed 444506 lines in 9.08075 secs.

In [8]:
## Check if the data is balanced 
data["is_click"].value_counts()

value,count
1.0,15128293
0.0,15126565


### Count Click Percent
In this section we will check the clicks distribution between the different columns (browser, placement id and more). We wanted to understand which features are important and to check for correlation between the features and the target column. 

#### By Browser

In [9]:
precnet_from_column(data, by = "browser_platform", value = "is_click")

browser_platform,is_click,count_click_per_class,count_total_obs,count_click_precent
DESK,0.0,14098935,26813831,0.5258083039309079
DESK,1.0,12714896,26813831,0.4741916960690921
PHON,1.0,2272753,3113298,0.7300146018787793
PHON,0.0,840545,3113298,0.26998539812122063
TBLT,0.0,165505,299215,0.5531306919773407
TBLT,1.0,133710,299215,0.4468693080226593
TV,0.0,21578,28512,0.756804152637486
TV,1.0,6934,28512,0.24319584736251404
OTHR,0.0,2,2,1.0


#### conclusions : 

*  DESK and TBLT have almost the same ratio of click ammount 50/50
*  PHONE and TV have almost the same ratio of click ammount 75/25 but they have the opisit behavior. in phonw we see 75 precent of click and in TV we see 75 precent of **not** click


#### By placement ID
We wanted to check whther a certain placement ID will reduce more clicks. 
Moreover, we wanted to check if all placement id's we have in the train exist in the test


In [10]:
precnet_from_placement_id = precnet_from_column(data, by = "placement_id_hash", value = "is_click")

In [11]:
precnet_from_placement_id = precnet_from_placement_id[precnet_from_placement_id["is_click"] == 1]
precnet_from_placement_id.sort("count_click_per_class", ascending = False )

is_click,placement_id_hash,count_click_per_class,count_total_obs,count_click_precent
1.0,f46a38c5c8d89917121df61f7a725748354c47dfc79cfe ...,605612,1551818,0.3902596825143155
1.0,39d46cdd8f08693dd0cb967d016efec97d0bc5eb1899dc ...,508630,661814,0.7685391968136063
1.0,3c1c8ea94e66caefc804eba418aa893f0d8fc361ad86e5 ...,469141,735340,0.6379919493023636
1.0,b1c0e2baa89e19c667a4c77835b2b1ccbbe87960f6a64b ...,437777,699592,0.6257604432297682
1.0,f1b3343d5e6491da91d7c65d1ad3140bb1983eedb1ed2d ...,419531,697863,0.6011652716937278
1.0,1fd7638e7057ac3ca2507f63cd5ac719375386f51a15b1 ...,365232,986810,0.3701138010356604
1.0,77de1750e42c8269e4f36cb4c4be23cef013ceab5cfc7a ...,343050,606445,0.5656737214421753
1.0,3d1a6e10ccba992a0cec1d9b67cd13b2033892ac6a6a43 ...,313418,413910,0.7572129206832403
1.0,41f1bf056030d77f4dd39c523e3218ac7796bb43c7c1eb ...,309416,464284,0.6664369222286359
1.0,932fbd7955800fd5a36f071a7e74f18343c263cf4bd950 ...,285079,357443,0.7975509381915439


In [12]:
count_click_precent = precnet_from_placement_id["count_click_precent"]
tc.visualization.histogram(count_click_precent)

Materializing SArray

## Time Analysis
In this section we wanted to examine the time columns in the data. We wanted to understand if we should split the data by time, and if the time of week/day impacts the users' clicks.  

After analyzing the data we witnessed that it's only from one month and that there is no difference between the days and times

In [13]:
precnet_from_placement_id = precnet_from_column(data, by = "day_of_week", value = "is_click")

In [14]:
precnet_from_placement_id_1 = precnet_from_placement_id[precnet_from_placement_id["is_click"] == 1]

In [15]:
precnet_from_placement_id

day_of_week,is_click,count_click_per_class,count_total_obs,count_click_precent
1,0.0,2675242,5189368,0.5155236629971126
2,0.0,2618713,5014554,0.522222514704199
1,1.0,2514126,5189368,0.48447633700288745
4,0.0,2512906,4836430,0.5195786975103537
2,1.0,2395841,5014554,0.477777485295801
5,1.0,2333005,4551970,0.5125264445943185
4,1.0,2323524,4836430,0.48042130248964626
3,0.0,2315396,4347651,0.5325625262929338
5,0.0,2218965,4551970,0.4874735554056815
3,1.0,2032255,4347651,0.4674374737070662


In [16]:
tc.show(precnet_from_placement_id_1["day_of_week"], precnet_from_placement_id_1["count_click_precent"])
# tc.visualization.histogram(precnet_from_placement_id[precnet_from_placement_id["is_click"] == 1]["count_click_precent"])

Materializing X axis SArray

Materializing Y axis SArray

In [17]:
data["day"] = data["page_view_start_time"].apply(lambda x : datetime.datetime.fromtimestamp(x/ 1000.0).month)
data["year"] = data["page_view_start_time"].apply(lambda x : datetime.datetime.fromtimestamp(x / 1000.0).year)

In [18]:
test["day"] = test["page_view_start_time"].apply(lambda x : datetime.datetime.fromtimestamp(x/ 1000.0).month)
test["year"] = test["page_view_start_time"].apply(lambda x : datetime.datetime.fromtimestamp(x / 1000.0).year)

# Model - Simple Turicrate Model

#### Data Preprocsing

In [19]:
delete_coulmns = ["gmt_offset","user_id_hash"]
features_list = ["target_item_taxonomy"]
features_categorical = ["target_id_hash","syndicator_id_hash","campaign_id_hash","placement_id_hash","publisher_id_hash","source_id_hash","source_item_type","browser_platform","country_code","region"]
label = "is_click"
time_column = "page_view_start_time"
precent = 0.8

In [20]:
data_filterd = data.remove_columns(delete_coulmns+features_list)
data_filterd = data_filterd.sort(time_column,ascending = True)
data_filterd[label] = data_filterd[label].astype(int)

#### Creating Turicreate Model

In [21]:
data_size = len(data_filterd)
train_size = int(precent * data_size)

data_filterd_train = data_filterd[:train_size]
data_filterd_test = data_filterd[train_size:]



In [22]:
model = tc.classifier.create(data_filterd_train, validation_set = data_filterd_test, target=label, verbose=1)

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


Logistic regression:

--------------------------------------------------------

Number of examples          : 24203886

Number of classes           : 2

...

| 7         | 18       | 0.925735  | 311.558829   | 0.763552          | 0.747347            |

| 8         | 23       | 0.922534  | 392.898283   | 0.763292          | 0.746931            |

| 9         | 24       | 1.000000  | 417.194179   | 0.762919          | 0.746394            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: LogisticClassifier              : 0.7571519088172941
PROGRESS: SVMClassifier                   : 0.7463936372536512
PROGRESS: ---------------------------------------------
PROGRESS: Selecting LogisticClassifier based on validation set performance.


In [23]:
predictions = model.predict(test, output_type = "probability")

In [24]:
predictions

dtype: float
Rows: 444506
[0.8714603024452857, 0.6868787468575411, 0.5047253275871896, 0.45844903324057124, 0.8461671889550321, 0.8902983410153349, 0.7332856356098257, 0.38025136289508427, 0.5657709384013792, 0.6919013940411637, 0.6983113630556186, 0.051924731326028796, 0.30057082335939495, 0.8631542717219076, 0.8042257467561831, 0.17270359727766887, 0.83811223966049, 0.4478179689946104, 0.3573753480644505, 0.06411812551777196, 0.24218316857640343, 0.873705591627583, 0.15492068088900282, 0.06778397982230663, 0.07941246689766107, 0.11455741863665518, 0.1086396738054453, 0.9220523389733023, 0.31843113860217603, 0.7154296205364675, 0.6160921539705312, 0.11148072102302943, 0.7276962865821008, 0.6541370050736948, 0.05042832565031878, 0.24920673088379858, 0.3765513014917416, 0.22344926209032642, 0.818355146164874, 0.5921335595741452, 0.5389237355661933, 0.2877680903448264, 0.4782790003687077, 0.08838830453379994, 0.7902131115819849, 0.18045678281838226, 0.975151815258386, 0.4772404299386263,

In [25]:
pred_df = pd.DataFrame(predictions)
pred_df.reset_index(inplace=True)
pred_df.columns = ['Id', "Predicted"]


In [26]:
pred_df

,Id,Predicted
0,0,0.871460
1,1,0.686879
2,2,0.504725
3,3,0.458449
4,4,0.846167
...,...,...
444501,444501,0.874733
444502,444502,0.589014
444503,444503,0.316266
444504,444504,0.676184


In [27]:
pred_df.to_csv('my_submission.csv', index = False)